<a href="https://colab.research.google.com/github/cedamusk/Astrophysics/blob/main/BBO_Detector_Calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
class BBODetector:
  def __init__(self):
    #Define the three arm vectore of the triangular configuration
    #Assuming equilateral triangle in x-y plane
    self.a_vec=np.array([1,0,0])
    self.b_vec=np.array([-0.5, np.sqrt(3)/2, 0])
    self.c_vec=np.array([-0.5, -np.sqrt(3)/2,0])

    #Reference point (center of triangle)
    self.x0=np.zeros(3)

  def normalized_frequency(self, f, L=1e6):
    """Convert frequency to normalized frequency fn=f/(2f*)
    where f*=1/(2pi*L) and L is arm length in meter"""
    f_star=1/(2*np.pi*L)
    return f/(2*f_star)

  def direction_vector(self, theta, phi):
    """Create direction vector from spherical coordinates"""
    return np.array([
        np.sin(theta)*np.cos(phi),
        np.sin(theta)*np.sin(phi),
        np.cos(theta)
    ])

  def sinc(self, X):
    """Calculate sinc(x)=sin(x)/x, handling x=0 case"""
    return np.sinc(X/np.pi) #numpy's sinc is define with pi

  def T_response(self, n_hat, f_n):
    """Calculate TT (symmetrized Sagnac) respons"""
    u_dot_n=np.dot(self.a_vec, n_hat) #Using a_vec as reference

    phase=np.exp(-1j*f_n/3*9) #Simplified phase term
    cos_term=(1+2 *np.cos(2*f_n))
    sinc_diff=(self.sinc(f_n *(1+u_dot_n))-
              self.sinc(f_n* (1-u_dot_n)))

    return phase* cos_term*sinc_diff/ (6*np.sqrt(3))

  def A_response(self, n_hat, f_n):
    """Calculate AA response for arm a"""
    a_dot_n=np.dot(self.a_vec, n_hat)
    bc_diff_dot_n=np.dot(self.c_vec-self.b_vec, n_hat)

    phase=np.exp(-1j *f_n/3*(6+bc_diff_dot_n))
    sin_term=-1j*np.sin(f_n)/(3*np.sqrt(2))
    sinc_sum=(self.sinc(f_n)/(3*np.sqrt(2))+
              np.exp(-2j*f_n)*self.sinc(f_n*(1-a_dot_n)))

    return sin_term*phase*sinc_sum

  def E_response(self, n_hat, f_n):
    """Calculate EE response for arm a"""
    a_dot_n=np.dot(self.a_vec, n_hat)

    phase=np.exp(-1j*f_n/3*6)
    basic_term=(self.sinc(f_n*(1+a_dot_n))+
                self.sinc(f_n*(1-a_dot_n)))

    return phase*basic_term / (6*np.sqrt(6))

  def calculate_response_pattern(self, f, response_type='T'):
    """Calculate response pattern over the sky"""
    theta=np.linspace(0, np.pi, 100)
    phi=np.linspace(0, 2*np.pi, 100)
    THETA, PHI=np.meshgrid(theta, phi)

    response=np.zeros_like(THETA, dtype=complex)
    f_n=self.normalized_frequency(f)

    for i in range(len(theta)):
      for j in range(len(phi)):
        n_hat=self.direction_vector(THETA[i, j], PHI[i, j])

        if response_type=="T":
          response[i, j]=self.T_response(n_hat, f_n)
        elif response_type=="A":
          response[i,j]=self.A_response(n_hat, f_n)
        elif response_type =="E":
          response[i, j]=self.E_response(n_hat, f_n)

    return THETA, PHI, np.abs(response)


  def plot_response_pattern(self, f, response_type='T'):
    """Plot the response pattern in 3D"""
    THETA, PHI, response=self.calculate_response_pattern(f, response_type)

    X=np.sin(THETA)*np.cos(PHI)
    Y=np.sin(THETA)*np.sin(PHI)
    Z=np.cos(THETA)

    fig=plt.figure(figsize=(10, 8))
    ax=fig.add_subplot(111, projection='3d')

    plot=ax.plot_surface(X,Y,Z, facecolors=plt.cm.viridis(response/response.max()))

    ax.set_title(f'{response_type}Response Pattern (f={f} Hz)')
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

    plt.colorbar(plt.cm.ScalarMappable(cmap=plt.cm.viridis), ax=ax,
                 label="Normalized Response Magnitude")
    plt.show()





In [ ]:
def main():
  detector=BBODetector()

  frequencies=[0.1, 1.0, 10.0]
  channels =['T', 'A', 'E']

  for f in frequencies:
    for channel in channels:
      detector.plot_response_pattern(f, channel)

  print("Response patterns plotted for all channels and frequencis")

In [ ]:
if __name__=="__main__":
  main()